<a href="https://colab.research.google.com/github/Gilbert9172/AI_Project/blob/main/n224a_model_selection_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 2 / SPRINT 2 / NOTE 4*

# 📝 Assignment
---

# 모델선택(Model Selection)

### 1) 캐글 대회를 이어서 진행합니다. RandomizedSearchCV 를 사용하여 하이퍼파라미터 튜닝을 진행합니다.

- [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)를 사용하세요.
- 분류문제에서 맞는 [scoring parameter](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values) metric을 사용하세요.
- [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) 사용을 권합니다.
- RandomizedSearchCV 를 사용해서 하이퍼파라미터 튜닝을 진행하고 최고 성능을 보이는 모델로 예측을 진행한 후 캐글에 제출합니다.
- **캐글 Leaderboard에서 개선된 본인 Score를 과제 제출폼에 제출하세요.**

In [1]:
pip install --upgrade category_encoders

     |████████████████████████████████| 81kB 4.0MB/s 


In [166]:
import pandas as pd
import numpy as np

target = 'vacc_h1n1_f'

train_data = pd.merge(pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'), 
                 pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[target], left_index=True, right_index=True)
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')

In [167]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train_data, train_size=0.80, test_size=0.20, 
                              stratify=train_data[target], random_state=2)

In [168]:
pd.set_option('mode.chained_assignment',  None)

def engineer(df):

    selected_cols = df.select_dtypes(include=['number', 'object'])
    labels = selected_cols.nunique() 
    selected_features = labels[labels <= 30].index.tolist() 
    df = df[selected_features]

    dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]
    df.drop(columns=dels, inplace=True)

    df.drop(columns = ['rent_own_r', 'inc_pov'], inplace=True)

    c = ['opinion_h1n1_vacc_effective','opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc','agegrp']
    ove = {"Dont Know":1, "Refused": 2, "Not At All Effective": 3, "Not Very Effective": 4, "Somewhat Effective ":5,"Very Effective":6}
    ohr = {"Dont Know":1, "Refused": 2, "Very Low": 3, "Somewhat Low": 4, "Somewhat High":5,"Very High":6}
    ohsfv = {"Dont Know":1, "Refused": 2, "Not At All Worried": 3, "Somewhat Worried": 4, "Not Very Worried":5,"Very Worried":6}
    agegap = {'6 Months - 9 Years':0,'10 - 17 Years':1,'18 - 34 Years':2,'35 - 44 Years':3,'45 - 54 Years':4,'55 - 64 Years':5,'65+ Years':6}

    df[c[0]] = df.opinion_h1n1_vacc_effective.map(ove)
    df[c[1]] = df.opinion_h1n1_risk.map(ohr)
    df[c[2]] = df.opinion_h1n1_sick_from_vacc.map(ohsfv)
    df[c[3]] = df.agegrp.map(agegap)

    return df

 
train = engineer(train) 
val = engineer(val)   
test = engineer(test)

In [169]:
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

X_test = test[features]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer  
from sklearn.pipeline import make_pipeline
from category_encoders import TargetEncoder
from scipy.stats import randint,uniform


pipe_ord = make_pipeline(
    TargetEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state = 0,
                           class_weight='balanced',
                           n_jobs = -1, 
                           oob_score = True)
)

params = {'randomforestclassifier__n_estimators': randint(100,300),
          'randomforestclassifier__max_depth': randint(5,20),
          'randomforestclassifier__min_samples_leaf': randint(2,20),
          'randomforestclassifier__min_samples_split' : randint(1,20),
          'randomforestclassifier__max_features' : randint(1,10)}
        #   'randomforestclassifier__random_state' : randint(1,10)
        #   }

R_CV = RandomizedSearchCV(pipe_ord,params,scoring="neg_mean_absolute_error",
                             n_jobs = 2, verbose=1,n_iter=50,cv=5)

R_CV.fit(X_train,y_train)

In [207]:
print('최적 하이퍼파라미터:', R_CV.best_params_)

최적 하이퍼파라미터: {'randomforestclassifier__max_depth': 17, 'randomforestclassifier__max_features': 7, 'randomforestclassifier__min_samples_leaf': 4, 'randomforestclassifier__min_samples_split': 9, 'randomforestclassifier__n_estimators': 245}


In [205]:
pipe_ord = make_pipeline(
    TargetEncoder(),
    SimpleImputer(),
    RandomForestClassifier(class_weight='balanced',
                           max_depth = 15,
                           max_features = 7,
                           min_samples_leaf = 4,
                           min_samples_split = 9,
                           n_estimators = 200,
                           random_state = 1,
                           n_jobs = -1, 
                           oob_score = True)
)

pipe_ord.fit(X_train,y_train)

y_v_pred = pipe_ord.predict(X_val)
print('학습정확도', pipe_ord.score(X_train,y_train))
print('검증정확도', pipe_ord.score(X_val,y_val))
print('F1_Score', f1_score(y_val, y_v_pred))

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


학습정확도 0.8695845565341161
검증정확도 0.8077333649626379
F1_Score 0.6270991488382792


In [ ]:
# sorted(pipe_ord.get_params().keys())

# 🔥 도전과제
### 2) [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 를 사용하여 하이퍼파라미터 튜닝을 진행합니다.
- 모델 성능을 높이기 위해 가능한 시도를 다 해보세요.
- 모델 성능에 가장 큰 영향을 준 하이퍼파라미터에 대해서 분석하고 설명해 보세요.



```
max_depth, 깊이가 깊어질수록 과적합 현상이 자주 발생한다.
```





In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer  
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from category_encoders import TargetEncoder

pipe_gr = make_pipeline(
    TargetEncoder(),
    SimpleImputer(strategy = 'mean'),
    RandomForestClassifier(random_state = 0,
                           class_weight='balanced', 
                           n_jobs = -1, 
                           oob_score = True)
)

param_grid = {
    'randomforestclassifier__n_estimators': [100,150,200],
    'randomforestclassifier__max_depth': [10,12,15],
    'randomforestclassifier__min_samples_leaf': [2,6,10],
    'randomforestclassifier__min_samples_split' : [2,6,8]
}

G_CV = GridSearchCV(pipe_gr,param_grid,scoring="neg_mean_absolute_error",
                             n_jobs =-1, verbose=1,cv=5)
# ,n_iter=50
G_CV.fit(X_train,y_train)

In [206]:
print('최적 하이퍼파라미터:', G_CV.best_params_)

최적 하이퍼파라미터: {'randomforestclassifier__max_depth': 15, 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__min_samples_split': 2, 'randomforestclassifier__n_estimators': 200}
